In [26]:
from boto.s3.connection import S3Connection
from ipyparallel import Client
import json
from collections import Counter
import pandas as pd
from operator import itervalues

ImportError: cannot import name itervalues

In [2]:
# Start your engines!
rc = Client()
engines = rc[:]
len(engines)

10

In [22]:
## ensure that every engine has the same functions available
with engines.sync_imports():
    from boto.s3.connection import S3Connection
    from collections import defaultdict
    import json


importing S3Connection from boto.s3.connection on engine(s)
importing defaultdict from collections on engine(s)
importing json on engine(s)


In [4]:
## connect to aws
s3 = S3Connection()
bucket = s3.get_bucket('twitter-streaming02062017')
list_of_aws_keys = [str(i.name) for i in bucket.list()]

In [23]:

%%px 
def twitter_scatter(individual_key):
    """Create a client to s3. Then, iterate through the keys in the given bucket. Return a counter of each hashtag."""
    s3 = S3Connection()
    bucket = s3.get_bucket('twitter-streaming02062017',validate=False)
    key_file = bucket.get_key(individual_key)
    strFile = key_file.get_contents_as_string()
    result = defaultdict(int)
    count = 0
    for tweet in strFile.split('/n'):
        try:
            entity = json.loads(tweet)
            try:
                entity = entity['entities']
                if entity != None and (entity['hashtags'] != None or len(entity['hashtags']) != 0):
                    for hashtags in entity['hashtags']:
                        result[hashtags['text']]+=1
            except KeyError :
                count+=1
        except ValueError:
            count+=1
    return result



In [24]:
print(len(list_of_aws_keys)) ## Number of files to go through
## scatter the keys to each engine
engines.scatter('s3' , list_of_aws_keys[:2]).get()
%px y = [twitter_scatter(_) for _ in s3]


95


In [28]:
## next, get the results of parsing the json from each engine
result = engines.gather('y')
from functools import reduce
top_results = Counter(list(result)[0])



In [32]:
#print out the top 100
list(result)

[defaultdict(int,
             {u'Canada': 1,
              u'\u0635\u062d\u064a\u062d_\u0645\u0633\u0644\u0645': 1,
              u'ChainedToTheRhythm': 19,
              u'carcrush': 1,
              u'PerHamtpon': 1,
              u'FtLauderdale': 1,
              u'PemimpinProfesional': 1,
              u'cyprus': 1,
              u'Hay\u0131r': 1,
              u'Liverpool': 1,
              u'calidadenuniformes': 1,
              u'tm1dn': 5,
              u'DressLikeAWoman': 1,
              u'Sprache': 1,
              u'cragpups': 1,
              u'LizaSoberano': 31,
              u'nikeforlife': 1,
              u'rodanandfields': 1,
              u'Xochimilco': 1,
              u'mlb': 1,
              u'Tecnolog\xeda': 1,
              u'\u0627\u062e\u0631\u062c\u0648\u0627_\u0639\u062a\u064a\u0628\u0647_\u0645\u0646_\u0643\u0648\u0643\u0628_\u0627\u0644\u0627\u0631\u0636': 9,
              u'breastcancer': 1,
              u'tcot': 1,
              u'FN': 1,
             

In [44]:
result = engines.gather('y')
from functools import reduce
product = reduce((lambda x, y: Counter(x)+Counter(y) ), result)

product.most_common()
names=['hastag','number']
produce_df = pd.DataFrame(product.most_common(),columns=names)

In [45]:
produce_df

,hastag,number
0,CD9,715
1,OneDBestFans,241
2,CamilaBestFans,198
3,iHeartAwards,120
4,사설토토사이트추천,110
5,BestFanArmy,96
6,KCAPinoyStar,93
7,izmirescort,80
8,LizaSoberano,68
9,ลุงวิศวะ,68
